# Exercise 3.0: Mesh plots (psyplot)
prepared by A. Lauber

Here we learn how to plot data from an unstructured grid like the [ICON](https://icon-model.org) grid as mesh grid.
There is the option to interpolate the data to a regular grid or to use the library [psyplot](https://psyplot.github.io), which was developed for plotting data on the unstructured grid.

Advantages of plotting on the unstructured grid:
- Shows the real output without interpolation
- No preprocessing of the data necessary

Disadvantages of plotting on the unstructured grid:
- It can be quite slow depending on the size of your NetCDF file
- Not all features are available yet (psyplot is still under development)


## Import libraries

In [ ]:
# Specific to our jupyterhub setup
import os
os.environ["PROJ_DATA"] = "/data/python_intro/miniconda/pkgs/proj-9.2.1-ha643af7_0"

In [ ]:
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cf
import cmcrameri.cm as cmc
import matplotlib.pyplot as plt
import xarray as xr

## Let's have a look into ICON data

For this exercise, we use a NetCDF file created with an ICON run by Nadja Omanovic. It was reduced to the 2m temperature over Switzerland to save space.

In [ ]:
ds = xr.open_dataset("../data/my_exp1_atm_3d_ml_20180921T000000Z_t2m.nc")
ds

### Exercise
Try plotting data with pcolormesh: Why does the following not work?

In [ ]:
f, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})

# Get data
lon, lat, temp = ds.clon, ds.clat, ds.t_2m[:, :, :]

h = ax.pcolormesh(lon, lat, temp, transform=ccrs.PlateCarree())

### Solution

`pcolormesh` cannot handle data on an unstructered grid. For plotting the data with pcolormesh, the data has to be remapped to a structured grid first.

## Plot data with pcolormesh using remapped data

The ICON output file we used before has been remapped using CDO. Let's have a look into the remapped data.

In [ ]:
ds_rmp = xr.open_dataset("../data/rmp_my_exp1_atm_3d_ml_20180921T000000Z.nc")
ds_rmp

### Exercise
 * Plot the 2m temperature over Switzerland
 * Use `projection=ccrs.Robinson()` (don't forget to transform the data)
 * Add the borders of Switzerland (hint `ax.add_feature(cartopy.feature.BORDERS`)
 * Cut off data outside of Switzerland (5.8<lon<10.7, 45.5<lat<48)
 * Add a colorbar
 * Set limits to the colorbar

In [ ]:
# get data
lon, lat, temp = ds_rmp.lon, ds_rmp.lat, ds_rmp.t_2m[0, 0, :, :]

# Code for plotting here

### Solution

In [ ]:
# get data
lon, lat, temp = ds_rmp.lon, ds_rmp.lat, ds_rmp.t_2m[0, 0, :, :]

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))

h = ax.pcolormesh(lon, lat, temp, transform=ccrs.PlateCarree(), vmin=275, vmax=295)

ax.set_title("2m temperature in Switzerland")
ax.add_feature(cartopy.feature.BORDERS)

ax.set_extent([5.8, 10.7, 45.5, 48])
plt.colorbar(h, ax=ax, extend='both')

### Exercise
Let's use a nicer colormap. The Python wrapper [cmcrameri](https://github.com/callumrollo/cmcrameri) was developed to use [scientific colour maps](https://www.fabiocrameri.ch/colourmaps/) in Python. Have a look at the available colour maps and choose one you like.

In [ ]:
# get data
lon, lat, temp = ds_rmp.lon, ds_rmp.lat, ds_rmp.t_2m[0, 0, :, :]

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))

h = ax.pcolormesh(lon, lat, temp, transform=ccrs.PlateCarree(), vmin=275, vmax=295)

ax.set_title("2m temperature in Switzerland")
ax.add_feature(cartopy.feature.BORDERS)

ax.set_extent([5.8, 10.7, 45.5, 48])
plt.colorbar(h, ax=ax, extend='both')

### Solution

In [ ]:
# get data
lon, lat, temp = ds_rmp.lon, ds_rmp.lat, ds_rmp.t_2m[0, 0, :, :]

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.Robinson()))

h = ax.pcolormesh(lon, lat, temp, transform=ccrs.PlateCarree(), vmin=275, vmax=295, cmap=cmc.nuuk)

ax.set_title("2m temperature in Switzerland")
ax.add_feature(cartopy.feature.BORDERS)

ax.set_extent([5.8, 10.7, 45.5, 48])
plt.colorbar(h, ax=ax, extend='both')

## Let's do the same on the original grid
We will now get started with [psyplot](https://psyplot.github.io). It has a similar syntax as other Python packages, e.g. `xr.open_dataset` --> `psy.open_dataset`. For doing map plots, we can use [psyplot.project.plot.mapplot](https://psyplot.github.io/psy-maps/generated/psyplot.project.plot.mapplot.html).

First we need to import the psyplot library:

In [ ]:
import psyplot.project as psy

We can change figures that have already been created. This is possible with the psyplot [update()](https://psyplot.github.io/psyplot/getting_started.html#controlling-the-update) option, but it requires changing the default settings in jupyter so that figures are not automatically closed. With this setting, we will need to manually close figures once we are done to avoid a mess.
To close the figure "ds", run `ds.close()`. To close all active figures, run `plt.close('all')`

In [ ]:
# The following is needed to show plots after they are being updated
%config InlineBackend.close_figures = False
psy.rcParams["auto_show"] = True

Let's now take the file with the original ICON grid and plot it.

## Load data with psyplot

In [ ]:
# Load dataset with psyplot
ds_icon = psy.open_dataset("../data/my_exp1_atm_3d_ml_20180921T000000Z_t2m.nc")
ds_icon

## Plot data with psyplot

In [ ]:
plot_icon = ds_icon.psy.plot.mapplot(name="t_2m")

I guess we could do that nicer....


### Exercise
In this exercise, you will create the same plot as before but on the unstructured ICON grid. You don't need to do the plot from scratch but you can use the update function `plot_icon.update(...)`, which works exactly like the `psy.plot.mapplot(...)` function, only that it is for updating the original plot and doesn't need all the input again. To check the available formatoptions, run `psy.plot.mapplot.keys()`. To get more information about a specific formatoption run `psy.plot.mapplot.docs('formatoption')` (check [psyplot.project.plot.mapplot](https://psyplot.github.io/psy-maps/generated/psyplot.project.plot.mapplot.html)).

 Use `plot_icon.update(...)` to adapt the plot in the following way: 
 * Use `projection=ccrs.Robinson()` (don't forget to transform the data)
 * Cut off data outside of Switzerland (5.8<lon<10.7, 45.5<lat<48)
 * Set limits to the colorbar
 * Use the colormap `cmc.nuuk`
 * Add a title
 
 (Hints: use `bounds={'method':'minmax','vmin':275,...}`)

In [ ]:
# Check available formatoptions
psy.plot.mapplot.keys()

In [ ]:
# Code here

### Solution

In [ ]:
plot_icon.update(
    projection=ccrs.Robinson(),
    transform=ccrs.PlateCarree(),
    map_extent=[5.8,10.7,45.5,48],
    bounds={'method':'minmax','vmin':275,'vmax':295},
    cmap=cmc.nuuk,
    title='2m temperature in Switzerland'
)

### Exercise
 * Add borders to the plot
 
Hint: the matplotlib axes can be accessed with `ax = plot_icon.plotters[0].ax`. Borders can then be added the same way as we added the lakes in exercise [ex2_0_intro_scatter.ipynb](ex2_0_intro_scatter.ipynb).

In [ ]:
# Add borders (don't forget to update the plot to see the plot)
# Code here

### Solution

In [ ]:
# Add borders (don't forget to update the plot to see the plot)
ax = plot_icon.plotters[0].ax
ax.add_feature(cf.BORDERS, edgecolor="0.1", zorder=100)
plot_icon.update()

Close the figure before creating a new one:

In [ ]:
plot_icon.close()

### Exercise formatoptions

Adding borders is still a bit complicated. A nicer way to add them is to use formatoptions, which you can generate yourself. The advantage is that they can be reused across different scripts. Psyplot is designed in a way that it allows users to easily create custom formatoptions. Check [the formatoption approach](https://psyplot.github.io/examples/general/example_extending_psyplot.html#3.-The-formatoption-approach) if you are interested.

MeteoSwiss and C2SM developed [iconarray](https://github.com/C2SM/iconarray), which contains various modules to facilitate working with ICON data. The package also includes some formatoptions like adding borders. Check out their [formatoptions](https://github.com/C2SM/iconarray#formatoptions) and repeat the exercise by using them. The package was developed together with [icon-vis](https://github.com/C2SM/icon-vis), which will be introduced in [Part3_icon-vis](../Part3_icon-vis).

In [ ]:
import iconarray

In [ ]:
# update code
# plot_icon = ds_icon.psy.plot.mapplot(...)

### Solution

In [ ]:
plot_icon = ds_icon.psy.plot.mapplot(
    name="t_2m",
    projection=ccrs.Robinson(),
    map_extent=[5.8, 10.7, 45.5, 48],
    cmap=cmc.nuuk,
    bounds={"method": "minmax", "vmin": 260, "vmax": 300},
    title="2m temperature Switzerland",
)

Note that [iconarray](https://github.com/C2SM/iconarray) shows borders and lakes by default when being imported. If you don't want that, you need to actively turn them off:

In [ ]:
plot_icon.update(lakes=False)